## Imports and Variables

In [1]:
import pandas
import numpy
import librosa
import requests
import seaborn
import sklearn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
labels = ['dishwasher', 'washing_machine', 'shower', 'washing_hands', 'flushing_toilet']

## Data Collection & Feature Extraction

In [3]:
# Get list of all recordings
data = requests.get("https://dolphin-app-9sdeq.ondigitalocean.app/api/v1/recordings").json()

recordings = []

for recording in data:
    recordings.append({
        "path": f'../recordings/combined/{recording["name"]}',
        "label": recording["label"],
        "environment": recording["environment"],
        "mfcc_mean": None,
        "mfcc_var": None,
        "n_samples": None,
        "chroma_mean": None,
        "chroma_var": None,
        "rms_mean": None,
        "rms_var": None,
        "sc_mean": None,
        "sc_var": None
    })

dataframe = pandas.json_normalize(recordings)

dataframe.head()

,path,label,environment,mfcc_mean,mfcc_var,n_samples,chroma_mean,chroma_var,rms_mean,rms_var,sc_mean,sc_var
0,../recordings/combined/a196dbae-1f50-4d0e-b345...,washing_hands,marsberg,None,None,None,None,None,None,None,None,None
1,../recordings/combined/826e36b1-2a7e-4d22-9c81...,washing_hands,marsberg,None,None,None,None,None,None,None,None,None
2,../recordings/combined/9a6b6b01-e466-47fb-9fcc...,washing_hands,marsberg,None,None,None,None,None,None,None,None,None
3,../recordings/combined/84ee02e6-44d5-4ab3-b875...,washing_hands,marsberg,None,None,None,None,None,None,None,None,None
4,../recordings/combined/ec24c55a-39ec-4b36-86fe...,washing_hands,marsberg,None,None,None,None,None,None,None,None,None


In [ ]:
# Function to extract features
def preprocess(dataframe):
    for index, row in tqdm(dataframe.iterrows()):
        audio, sample_rate = librosa.load(row["path"])
        
        # Calculate mean and variance of the MFCCs
        mfcc_stft = librosa.feature.mfcc(y=audio, sr=sample_rate)
        dataframe.at[index, "mfcc_mean"] = numpy.mean(mfcc_stft)
        dataframe.at[index, "mfcc_var"] = numpy.var(mfcc_stft)
        
        # Calculate number of samples, representing length
        dataframe.at[index, "n_samples"] = audio.shape[0]

        # Calculate mean and variance of the chromagram
        chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        dataframe.at[index, "chroma_mean"] = numpy.mean(chroma_stft)
        dataframe.at[index, "chroma_var"] = numpy.var(chroma_stft)
        
        # Calculate mean and variance of the root mean square array
        rms = librosa.feature.rms(y=audio)
        dataframe.at[index, "rms_mean"] = numpy.mean(rms)
        dataframe.at[index, "rms_var"] = numpy.var(rms)
        
        # Calculate mean and variance of the spectral centroid
        spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sample_rate)
        dataframe.at[index, "sc_mean"] = numpy.mean(spectral_centroid)
        dataframe.at[index, "sc_var"] = numpy.var(spectral_centroid)
        
    return dataframe

dataframe = preprocess(dataframe)

117it [03:41,  6.07s/it]

In [ ]:
# Extract separate dataframes for environments marsberg and duesseldorf
dataframe_marsberg = dataframe[dataframe["environment"] == "marsberg"]
dataframe_duesseldorf = dataframe[dataframe["environment"] == "duesseldorf"]

## Support Vector Classifier trained on data from _marsberg_

In [ ]:
# Prepare and split data for training
y = dataframe_marsberg['label']
x = dataframe_marsberg.drop(['label', 'path', 'environment'], axis=1)

y.replace(labels,[0, 1, 2, 3, 4], inplace=True)

cols = x.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x)
x = pandas.DataFrame(np_scaled, columns = cols)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
model = SVC(
    kernel="rbf",
    gamma="scale",
    probability=True
).fit(x_train, y_train)

In [ ]:
test_predictions = model.predict(x_test)
print(accuracy_score(y_test, test_predictions))

In [ ]:
def confusion_matrix_table(y_true, y_pred):
    labels = sklearn.utils.multiclass.unique_labels(y_test)
    columns = [f'{label}' for label in labels]
    index = [f'{label}' for label in labels]
    confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=labels)
    table = pandas.DataFrame(confusion_matrix, columns=columns, index=index)
    return table

def confusion_matrix_plot(y_true, y_pred):
    table = confusion_matrix_table(y_true, y_pred)
    plot = seaborn.heatmap(table, annot=True, fmt="d", cmap="viridis")
    plt.yticks(rotation=0)
    plt.ylabel(
        'GROUND TRUTH', 
        fontsize = 12, 
        fontweight = 600
    )
    plt.xlabel(
        'PREDICTED LABEL', 
        fontsize = 12, 
        fontweight = 600
    )
    return plot

confusion_matrix_plot(y_test, test_predictions)

In [ ]:
# Calculate accuracy on data from duesseldorf
y_duesseldorf = dataframe_duesseldorf['label']
x_duesseldorf = dataframe_duesseldorf.drop(['label', 'path', 'environment'], axis=1)

y_duesseldorf.replace(labels,[0, 1, 2, 3, 4], inplace=True)

cols = x_duesseldorf.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x_duesseldorf)
x_duesseldorf = pandas.DataFrame(np_scaled, columns = cols)

test_duesseldorf_predictions = model.predict(x_duesseldorf)
print(accuracy_score(y_duesseldorf, test_duesseldorf_predictions))

In [ ]:
confusion_matrix_plot(y_duesseldorf, test_duesseldorf_predictions)

## Support Vector Classifier trained on data from all environments

In [ ]:
# Prepare and split data for training
y = dataframe['label']
x = dataframe.drop(['label', 'path', 'environment'], axis=1)

y.replace(labels,[0, 1, 2, 3, 4], inplace=True)

cols = x.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(x)
x = pandas.DataFrame(np_scaled, columns = cols)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [ ]:
model = SVC(
    kernel="rbf",
    gamma="scale",
    probability=True
).fit(x_train, y_train)

In [ ]:
test_predictions = model.predict(x_test)
print(accuracy_score(y_test, test_predictions))

In [ ]:
def confusion_matrix_table(y_true, y_pred):
    labels = sklearn.utils.multiclass.unique_labels(y_test)
    columns = [f'{label}' for label in labels]
    index = [f'{label}' for label in labels]
    confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred, labels=labels)
    table = pandas.DataFrame(confusion_matrix, columns=columns, index=index)
    return table

def confusion_matrix_plot(y_true, y_pred):
    table = confusion_matrix_table(y_true, y_pred)
    plot = seaborn.heatmap(table, annot=True, fmt="d", cmap="viridis")
    plt.yticks(rotation=0)
    plt.ylabel(
        'GROUND TRUTH', 
        fontsize = 12, 
        fontweight = 600
    )
    plt.xlabel(
        'PREDICTED LABEL', 
        fontsize = 12, 
        fontweight = 600
    )
    return plot

confusion_matrix_plot(y_test, test_predictions)